In [ ]:
%matplotlib inline
%load_ext autoreload

In [ ]:
%autoreload 2

In [ ]:
import os
import time
import datetime
import warnings
import matplotlib
matplotlib.rcParams.update({'font.size': 16})
import pandas
import numpy
import scipy.stats
import abcutils

## Load and Synthesize Data from CSV

This process loads each summary CSV file, creates a few derived metrics, and then merges each system's CSV into a single global dataset that can be sliced and diced by system, benchmark, or any other way.  We are now caching the processed CSV in HDF5 format to speed up initial data ingest at the beginning of each analysis.  Delete the `CACHE_FILE` to re-generate this cache (e.g., when the contents of the CSV are updated).

In [ ]:
CACHE_FILE = 'cache.hdf5'
if CACHE_FILE and os.path.isfile(CACHE_FILE):
    print "Loading from cache %s" % CACHE_FILE
    df = pandas.read_hdf(CACHE_FILE, 'summary')
else:
    df = pandas.concat([abcutils.load_and_synthesize_csv('summaries/edison-summaries_2017-02-14-2018-02-28.csv', system='edison'),
                        abcutils.load_and_synthesize_csv('summaries/cori-summaries_2017-02-14-2018-02-28.csv', system='cori'),
                        abcutils.load_and_synthesize_csv('summaries/alcf-tokio-results-2_14_17-2_15_18.csv', system='mira')],
                       axis='rows')
    if CACHE_FILE:
        df.to_hdf(CACHE_FILE, key='summary', mode='w', format='fixed', complevel=9, complib='zlib')
        print "Cached synthesized CSV to %s" % CACHE_FILE
    
# Reset the index to ensure that there are no degenerate indices in the final dataframe
df.index = pandas.Index(data=numpy.arange(len(df)), dtype='int64')

In [ ]:
pd2epoch = lambda x: time.mktime(x.to_pydatetime().timetuple())

In [ ]:
group_by = ['_test_platform', '_benchmark_id']

## Demonstrate a Single Test Platform

Look at one combination of (compute system, file system, benchmark) to show what this UMAMI analysis can do.

### Define Input Parameters

In [ ]:
# test_platform = 'scratch2@edison'
test_platform = 'cscratch@cori-knl'
# test_platform = 'cscratch@cori-haswell'
# test_platform = 'mira-fs1@mira'

# benchmark_id = 'ior_fpp_write'
# benchmark_id = 'dbscan_read_shared_read'
benchmark_id = 'vpicio_uni_shared_write'
# benchmark_id = 'ior_shared_write'
# benchmark_id = 'hacc_io_read_fpp_read'

plot_metric = 'darshan_agg_perf_by_slowest_posix_gibs'
date_start = datetime.datetime(2017, 2, 14)
date_end = datetime.datetime(2018, 3, 1)
group_by = ['_test_platform', '_benchmark_id']
delta = datetime.timedelta(days=1).total_seconds()

filtered_df = df.groupby(by=group_by).get_group((test_platform, benchmark_id))
filtered_df = filtered_df[filtered_df['darshan_total_gibs_posix'] > 1.0]
filtered_df = filtered_df[filtered_df['_datetime_start'] < date_end]
filtered_df = filtered_df[filtered_df['_datetime_start'] >= date_start]

print "test_platform =", test_platform
print "benchmark_id =", abcutils.CONFIG['benchmark_labels'].get(benchmark_id, benchmark_id)
print "plot_metric =", abcutils.CONFIG['metric_labels'].get(plot_metric, plot_metric)
print "date_start =", date_start.isoformat()
print "date_end =", date_end.isoformat()

In [ ]:
# Width of simple moving average (SMA) short/long windows
short_window = 7
long_window = 28

print "Short window will average over %d measurements at a time" % short_window
print "Long window will average over %d measurements at a time" % long_window

In [ ]:
# Metrics to include in UMAMI renderings and analysis.  Anything that
# _might_ affect performance should be included here.
umami_rows = [
    'darshan_agg_perf_by_slowest_posix_gibs',
    'coverage_factor_bw',
    'coverage_factor_nodehrs',
    'fs_ave_mds_cpu',
    'fs_tot_metadata_ops',
    'fs_ave_oss_cpu',
    'fs_tot_open_ops',
    'fshealth_ost_most_full_pct',
    'fshealth_ost_overloaded_oss_count',
    'jobsdb_concurrent_nodes',
    'topology_job_max_radius',
]

## Calculate Simple Moving Averages (SMAs)

Compare a short-window SMA and a long-window SMA and use the places where they cross over to divide the entire year into _regions_ of interesting benchmark behavior.

In [ ]:
sma_intercepts = abcutils.features.sma_intercepts(filtered_df, plot_metric, short_window, long_window)
sma_minmax = abcutils.features.sma_local_minmax(filtered_df,
                                                plot_metric,
                                                short_window,
                                                long_window,
                                                min_domain=7,
                                                max_func=pandas.Series.idxmin)

For each region defined above, find the _minimum_ performance observed and denote that measurement (and its associated job) as a _locus_.  We then collate all _loci_ into a set of poorly performing benchmarks that are worth contextualizing with UMAMI.

We also plot the raw performance data (light blue bars), the short SMA (orange line), the long SMA (green line), and all loci (red bars) to visually verify that the loci we've identified are indeed poorly performing jobs.

In [ ]:
x_raw = filtered_df['_datetime_start'].apply(lambda x: time.mktime(x.timetuple()))
y_raw = filtered_df[plot_metric]

loci_df = filtered_df.loc[sma_minmax.index]
x_low = sma_minmax['_datetime_start'].apply(lambda x: time.mktime(x.timetuple()))
y_low = sma_minmax[plot_metric]
print "Found %d loci across %s" % (len(loci_df), date_end - date_start)

### plot the raw data
fig, ax = matplotlib.pyplot.subplots()
fig.set_size_inches(16, 4)
ax.grid()
ax.bar(x_raw, y_raw, width=delta, alpha=0.5)
ax.bar(x_low, y_low, width=delta, color='red')
ax.set_ylabel(abcutils.CONFIG['metric_labels'].get(plot_metric, plot_metric).replace(" ", "\n"))
ax.set_xticklabels([datetime.datetime.fromtimestamp(x).strftime("%b %d") for x in ax.get_xticks()])
fig.suptitle("%s on %s" % (abcutils.CONFIG['benchmark_labels'].get(benchmark_id, benchmark_id),
                           test_platform))

### also calculate and plot the SMAs
sma_short = abcutils.features.calculate_sma(filtered_df, '_datetime_start', plot_metric, short_window)
sma_long = abcutils.features.calculate_sma(filtered_df, '_datetime_start', plot_metric, long_window)

### plot the intercept points demarcating different regions
#intercepts = numpy.array([(filtered_df['_datetime_start'].loc[x], filtered_df[plot_metric].loc[x]) for x in sma_intercepts.index])
#x_intercept = [(x - numpy.datetime64('1970-01-01T00:00:00Z')) / numpy.timedelta64(1, 's') for x in intercepts[:, 0]]
#y_intercept = intercepts[:, 1]
#ax.scatter(x_intercept, y_intercept, color='red', marker='.')

x_sma_short = [pd2epoch(x) for x in sma_short.index]
y_sma_short = sma_short.values

x_sma_long = [pd2epoch(x) for x in sma_long.index]
y_sma_long = sma_long.values

ax.plot(x_sma_short, y_sma_short, color='C1', linewidth=2)
ax.plot(x_sma_long, y_sma_long, color='C2', linewidth=2)

pass

## Generate UMAMI Diagrams Around Loci

Generate UMAMI diagrams that _end_ at each locus and have `long_window` days' of benchmark data preceding them.  Don't bother creating UMAMI diagrams for benchmarks with fewer than `short_window` benchmark data in the preceding `long_window` days.

Note that this process mixes up the semantic meaning of `long_window`.  When defining loci, `long_window` refers to a number of benchmark measurements, not days.  Ideally, one benchmark runs each day so this semantic difference is trivial.  However in reality, there are days when no benchmarks are run meaning loci are defined using a series of `long_window` benchmark measurements that often span _more than_ `long_window` days.

Practically speaking, this does not change very much as long as the ratio of `long_window` in days to `long_window` in benchmark measurements is close to 1.0.

In [ ]:
max_renders = 1

print "Rendering a maximum of %d UMAMI diagrams" % max_renders

rendered = 0
for row in sma_minmax.itertuples():
    date_filter = filtered_df['_datetime_start'] <= row[1]
    date_filter &= filtered_df['_datetime_start'] >= row[1] - datetime.timedelta(days=long_window)
    umami_region = filtered_df[date_filter]
    if len(umami_region) >= short_window:
        abcutils.plot.generate_umami(umami_region, umami_rows)
        rendered += 1
        if rendered == max_renders:
            break
    else:
        print "Skipping locus at %s because it has only %d data points (%d required)" % (umami_region['_datetime_start'], len(umami_region), short_window)

## Tabulate frequency of different problems

Because we define loci to be local minima (i.e., the worst benchmark measured in a temporally local region), all of the UMAMIs we generated above _should_ end on an extremely bad day.  To automatically identify the possible causes for bad performance at each locus, we look at all of the UMAMI metrics and flag those that also ended on extremely poor (e.g., worst quartile) values.  This is exactly the same process we used in the PDSW'17 paper's case studies, but now we have automated the process.

With this method of flagging, we keep a running total of metrics that were flagged as possible culprits as we examine each locus.  Note that multiple metrics can be flagged for a single locus (e.g., low coverage factor _and_ high MDS load can both be flagged for a single benchmark run), so the sum of flags over all metrics will usually add up to more than the total number of loci.

In [ ]:
def classify_root_cause(dataframe, dependent_column):
    causes = []
    for column in dataframe.columns:
        big_is_good = abcutils.CONFIG['metric_big_is_good'].get(column, True)

        result = None
        if big_is_good:
            cutoff = numpy.nanpercentile(dataframe[column].iloc[0:-1], 25)
            if dataframe[column].iloc[-1] < cutoff:
                result = (column, dataframe[column].iloc[-1], "<", cutoff)
        else:
            cutoff = numpy.nanpercentile(dataframe[column].iloc[0:-1], 75)
            if dataframe[column].iloc[-1] > cutoff:
                result = (column, dataframe[column].iloc[-1], ">", cutoff)
        
        if column == dependent_column:
            if result is None:
                warnings.warn("%s (index %s) not in the worst quartile of %d values" % (column, dataframe[column].index[-1], len(dataframe)))
#               print dataframe[dependent_column]
                return []
        elif result:
            causes.append(result)
                
    return causes

In [ ]:
results = {}

for row in sma_minmax.itertuples():
    # look for rows indicating an extreme value that are preceded by a region boundary
    date_filter = filtered_df['_datetime_start'] <= row[1]
    date_filter &= filtered_df['_datetime_start'] >= row[1] - datetime.timedelta(days=long_window)
    umami_region = filtered_df[date_filter][['_datetime_start'] + umami_rows]
    if len(umami_region) >= short_window:
        results['num_loci'] = results.get('num_loci', 0) + 1
        for cause in classify_root_cause(umami_region[umami_rows], plot_metric):
            results[cause[0]] = results.get(cause[0], 0) + 1
num_loci = results.pop('num_loci')

print "Number of times each metric was flagged across %d loci:" % num_loci
for key in reversed(sorted(results.keys(), key=lambda x: results[x])):
    print "%3d %s" % (results[key], abcutils.CONFIG['metric_labels'].get(key, key))

## Calculate root causes over all tests

We now apply the above analysis to the entirety of the benchmark data across all systems.

Note that warnings about certain loci not being in the worst quartile indicate that the SMA-based method we use to identify local minima is not perfect.  There are a variety of other methods (including some canned algorithms) that we can swap in to improve our classification of loci.

In [ ]:
# Global plot parameters
plot_metric = 'darshan_agg_perf_by_slowest_posix_gibs'
date_start = datetime.datetime(2017, 2, 14)
date_end = datetime.datetime(2018, 3, 1)
group_by = ['_test_platform', '_benchmark_id']

# Determine which plots to generate
test_platforms = sorted(df['_test_platform'].unique())
benchmark_ids = sorted(df['_benchmark_id'].unique())

# test_platforms = ['cscratch@cori-knl']
# benchmark_ids = ['dbscan_read_shared_read', 'vpicio_uni_shared_write']

print "plot_metric =", abcutils.CONFIG['metric_labels'].get(plot_metric, plot_metric)
print "date_start =", date_start.isoformat()
print "date_end =", date_end.isoformat()

grouped_df = df.groupby(by=group_by)

results = {}
for test_platform in test_platforms:
    for benchmark_id in benchmark_ids:
        try:
            filtered_df = grouped_df.get_group((test_platform, benchmark_id))
        except KeyError:
            continue
        filtered_df = filtered_df[filtered_df['darshan_total_gibs_posix'] > 1.0]
        filtered_df = filtered_df[filtered_df['_datetime_start'] < date_end]
        filtered_df = filtered_df[filtered_df['_datetime_start'] >= date_start]

        sma_minmax = abcutils.features.sma_local_minmax(filtered_df,
                                                        plot_metric,
                                                        short_window,
                                                        long_window,
                                                        min_domain=short_window,
                                                        max_func=pandas.Series.idxmin)
#       print "Found %d loci for %s, %s" % (len(sma_minmax), test_platform, benchmark_id)
        for row in sma_minmax.itertuples():
            date_filter = filtered_df['_datetime_start'] <= row[1]
            date_filter &= filtered_df['_datetime_start'] >= row[1] - datetime.timedelta(days=long_window)
            umami_region = filtered_df[date_filter][['_datetime_start'] + umami_rows]
            if len(umami_region) >= short_window:
                results['num_loci'] = results.get('num_loci', 0) + 1
                for cause in classify_root_cause(umami_region[umami_rows], plot_metric):
                    results[cause[0]] = results.get(cause[0], 0) + 1
                    
num_loci = results.pop('num_loci')

The following bar graph shows the total number of times each metric has been flagged as a possible source of performance loss as defined above: its value was "bad" coincident with each locus, where a locus is a job whose performance was abnormally poor and "bad" is defined as being within the 25th worst percentile.

In [ ]:
fig, ax = matplotlib.pyplot.subplots()
fig.set_size_inches(16,4)

x_labels = list(reversed(sorted(results.keys(), key=lambda x: results[x])))
y_values = [results[x] for x in x_labels]
x_values = numpy.arange(len(y_values))

ax.yaxis.grid(True)
ax.bar(x_values, y_values)
ax.set_xticks(x_values)
ax.set_xticklabels([abcutils.CONFIG['metric_labels'].get(x, x) for x in x_labels], rotation=45, ha='right')
ax.set_ylabel("Number of occurrences")
ax.set_title("Hypothesized Causes of Performance Degradation (%d Jobs Total)" % num_loci)

for index, x_value in enumerate(x_values):
    ax.annotate("%d" % y_values[index], xy=(x_value, y_values[index]), ha='center')
            
pass

There are many caveats with the above plot; notably, the majority of jobs were run on Lustre since this data includes all Edison, Cori+KNL, and Cori+Haswell jobs.  In addition, the Mira data does not currently contain file system health data (although it is available), so the "Number of Overloaded OSSes" may be underreported.

The most appropriate way to present this data is to produce one bar graph per test platform (compute system + file system combination) so that metrics that are only available on one test platform aren't being directly compared with others that are.